In [85]:
import pandas as pd
import numpy as np

In [86]:
temp_df = pd.read_csv("IMDB Dataset.csv")

In [87]:
df = temp_df.iloc[:10000]

In [88]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [89]:
df['sentiment'].value_counts()

sentiment
positive    5028
negative    4972
Name: count, dtype: int64

In [90]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [91]:
df.duplicated().sum()

17

In [92]:
df = df.drop_duplicates()

In [93]:
import re 
def remove_tags(text):
    new_text = re.sub(re.compile('<.*?>'), '', text)
    return new_text

In [94]:
df.loc[:, 'review'] = df['review'].apply(remove_tags)

In [95]:
df.loc[:,'review'] = df['review'].apply(lambda x:x.lower())

In [96]:
from nltk.corpus import stopwords
def remove_stopword(text):
    new_text = []
    for w in text.split():
        if w not in stopwords.words('english'):
            new_text.append(w)
    return " ".join(new_text)
df.loc[:,'review'] = df['review'].apply(remove_stopword)

In [97]:
df.head()

,review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,positive
1,wonderful little production. filming technique...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically there's family little boy (jake) thi...,negative
4,"petter mattei's ""love time money"" visually stu...",positive


In [98]:
X = df.iloc[:,0:1]
y = df['sentiment']

In [99]:
X

,review
0,one reviewers mentioned watching 1 oz episode ...
1,wonderful little production. filming technique...
2,thought wonderful way spend time hot summer we...
3,basically there's family little boy (jake) thi...
4,"petter mattei's ""love time money"" visually stu..."
...,...
9995,"fun, entertaining movie wwii german spy (julie..."
9996,"give break. anyone say ""good hockey movie""? kn..."
9997,movie bad movie. watching endless series bad h...
9998,"movie probably made entertain middle school, e..."


In [100]:
y

0       positive
1       positive
2       positive
3       negative
4       positive
          ...   
9995    positive
9996    negative
9997    negative
9998    negative
9999    positive
Name: sentiment, Length: 9983, dtype: object

In [101]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [102]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [103]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [104]:
X_train.shape

(7986, 1)

# Apply BOW

In [105]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [106]:
X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.fit_transform(X_test['review']).toarray()

In [109]:
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.ensemble import RandomForestClassifier

In [111]:
rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.5448172258387581

In [110]:
cv = CountVectorizer(max_features=3000)
# using max_features will give most used 3000 words

X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.fit_transform(X_test['review']).toarray()

rf = RandomForestClassifier()
rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.542814221331998

# Apply Word-2-vec

In [114]:
import gensim

In [116]:
from gensim.models import Word2Vec,KeyedVectors
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [117]:
story = []
for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [119]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [120]:
model.build_vocab(story)

In [121]:
model.train(story,total_examples=model.corpus_count,epochs=model.epochs)

(5876095, 6212140)

In [122]:
len(model.wv.index_to_key)

31845

In [124]:
def document_vector(doc):
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc],axis=0)

In [125]:
from tqdm import tqdm

In [126]:
X = []
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))

100%|██████████| 9983/9983 [04:35<00:00, 36.26it/s]


In [127]:
X = np.array(X)

In [128]:
X.shape

(9983, 100)

In [129]:
X_train,X_train,y_train,y_train = train_test_split(X,y,test_size=0.2,random_state=1)

In [130]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred=rf.predict(X_test)
accuracy_score(y_test,y_pred)

c:\Users\tusha\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


ValueError: could not convert string to float: '8 simple rules dating teenage daughter auspicious start. supremely-talented tom shadyac involved project. meant comedy would nothing less spectacular, that\'s exactly happened: show remains one freshest, funniest, wittiest shows made long time. every line, facial expression, casting choice, scene, wreaked perfection. one episode thought, "man good rest". one standout. again, kind perfectionism we\'ve come expect tom. know, tom shadyac director ace ventura (first movie), nutty professor (first one) liar liar. quite résumé. he\'s producer director, magic touch felt every episode.the family consists of:the father: paul hennessy (john ritter): nice, slightly neurotic, pushover time time, works sports writer. john unfortunately passed away 2003 leaving fond memory near-sure cancellation contemplations suits.the mother: cate (katey sagal): come on, fall love katey played peg married children? al bundy hero. viewers gave respect love never had. 